## KeyPhrase extraction
* Use SemEval 2010 dataset - [H49](https://github.com/boudinfl/ake-datasets/blob/master/datasets/SemEval-2010/train/H-49.xml) dataset for TF-IDF vectorization
* Use SemEval 2010 dataset - [H7](https://github.com/boudinfl/ake-datasets/blob/master/datasets/SemEval-2010/test/H-7.xml) for inference
* Evaluation of the results shows precisoin, recall, f1, and precision@5 per each document and also the mean of those

In [1]:
import operator
import json
import numpy as np
from pathlib import Path
from glob import glob
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from xml.etree import ElementTree

from tqdm import tqdm_notebook as tqdm

In [2]:
def read(directory):
    docs = {}
    for doc_path in tqdm(glob(f'{directory}/*.xml')):
        doc = ElementTree.parse(doc_path)
        sentences = []
        for sentence in doc.find('document').find('sentences').findall('sentence'):
            sentences.append(' '.join([token.find('lemma').text.lower() 
                                       for token in sentence.find('tokens').findall('token')]))

        docs[doc_path.split('/')[-1].split('.')[0]] = '\n'.join(sentences)
    return docs

In [ ]:
train_sentences = read('ake-datasets/datasets/SemEval-2010/train')
test_sentences = read('ake-datasets/datasets/SemEval-2010/test')
len(train_sentences), len(test_sentences)

In [ ]:
vectorizer = TfidfVectorizer(lowercase=True, stop_words='english', ngram_range=(1, 3))
trainvec = vectorizer.fit_transform(train_sentences.values())
feature_names = vectorizer.get_feature_names()

In [ ]:
with open('ake-datasets/datasets/SemEval-2010/references/test.author.stem.json', 'r') as f:
    target = json.load(f)
    target = {doc_name: [k[0] for k in keyphrases] for doc_name, keyphrases in target.items()}

In [ ]:
target['H-11']

In [ ]:
print(test_sentences['H-11'])

In [ ]:
def extract_keyphrases(vec, feature_names, nb_keywords=5):
    feature_index = vec.nonzero()[1]
    tfidf_scores = zip(feature_index, [vec[0, x] for x in feature_index])
    # Scale scores by n-gram length
    scores = {feature_names[i]: s * len(feature_names[i].split()) for i, s in tfidf_scores}
    scores = sorted(scores.items(), key=operator.itemgetter(1), reverse=True)[:nb_keywords]
    return [keyphrase for keyphrase, score in scores]

In [ ]:
predictions = {}
for doc_id, doc in test_sentences.items():
    vec = vectorizer.transform([doc])[0]
    keyphrases = extract_keyphrases(vec, feature_names=feature_names, nb_keywords=5)
    predictions[doc_id] = keyphrases

In [ ]:
predictions['H-11'], target['H-11']

In [ ]:
precision, recall, f1, precision_5 = [], [], [], []
for doc_id in sorted(predictions.keys()):
    p = set(predictions[doc_id])
    t = set(target[doc_id])
    at_5 = set(target[doc_id][:5])

    # We always predict 5 keywords
    precision.append(len(p.intersection(t)) / len(p))
    recall.append(len(p.intersection(t)) / len(t))
    f1.append(0 if precision[-1] + recall[-1] == 0 else 2 * precision[-1] * recall[-1] / (precision[-1] + recall[-1]))
    precision_5.append(len(p.intersection(at_5)) / len(p))
    print(f'{doc_id:5} -> Precision: {precision[-1]:.2f} Recall: {recall[-1]:.2f} F1: {f1[-1]:.2f} precision@5: {precision_5[-1]:.2f}')

print()
print('--------------Mean-------------')
print(f'Precision: {np.mean(precision):.2f} Recall: {np.mean(recall):.2f} F1: {np.mean(f1):.2f}   precision@5: {np.mean(precision_5):.2f}')